<a href="https://colab.research.google.com/github/sanya-shetty/cseGIT/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python
!pip install tensorflow

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Set dataset directories (update the paths if you're using Google Drive)
train_dir = '/content/drive/MyDrive/Fake_Brand_Detection/dataset/train/'
valid_dir = '/content/drive/MyDrive/Fake_Brand_Detection/dataset/valid/'

# ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'  # 0 for fake, 1 for genuine
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

Found 12 images belonging to 2 classes.
Found 12 images belonging to 2 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Create a CNN model
def create_cnn_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Output: 0 for fake, 1 for geniune
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create and summarize the model
model = create_cnn_model()
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │       1,605,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,699,009 (6.48 MB)

 Trainable params: 1,699,009 (6.48 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the CNN model
model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator
)

# Save the trained model for later use
model.save('/content/brand_model.h5')

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.4167 - loss: 0.7032 - val_accuracy: 0.5833 - val_loss: 1.4650
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5833 - loss: 1.4650 - val_accuracy: 0.8333 - val_loss: 0.5191
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.8333 - loss: 0.5191 - val_accuracy: 0.4167 - val_loss: 0.7504
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.4167 - loss: 0.7504 - val_accuracy: 0.6667 - val_loss: 0.5876
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6667 - loss: 0.5876 - val_accuracy: 1.0000 - val_loss: 0.4770
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.4770 - val_accuracy: 0.9167 - val_loss: 0.4033
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.9167 - loss: 0.4033 - val_accuracy: 0.9167 - val_loss: 0.3427
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.9167 - loss: 0.3427 - val_accuracy: 0.9167 - val_loss: 0.2781
Epoch 9/10
1/1 ━━━━━━━━━━━━

In [ ]:


import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('/content/brand_model.h5')

# Function to preprocess the image for prediction
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Function to predict fake or genuine
def detect_fake_or_genuine(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)

    if prediction[0] < 0.5:
        print("This product is fake!")
    else:
        print("This product is genuine.")

# Test with an image
test_image_path = '/content/drive/MyDrive/Fake_Brand_Detection/adidasori.jpg'
detect_fake_or_genuine(test_image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
This product is genuine.


In [ ]:
# Step 1: Install Gradio
!pip install gradio



# Step 2: Import necessary libraries
import gradio as gr
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image
# Import the HTML display class
from IPython.display import HTML, display

# Step 3: Load the trained model
model = tf.keras.models.load_model('/content/brand_model.h5')

# Step 4: Preprocess the image before making a prediction
def preprocess_image(image):
    img = np.array(image)
    img = cv2.resize(img, (128, 128))  # Resize to the model's expected input size
    img = img.astype('float32') / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension for model
    return img

# Step 5: Function to predict fake or genuine
def detect_fake_or_genuine(image):
    img = preprocess_image(image)
    prediction = model.predict(img)

    if prediction[0] <0.5:
        return "This product is fake!"
    else:
        return "This product is genuine."

interface = gr.Interface(
    fn=detect_fake_or_genuine,  # The function to run on input
    inputs=gr.Image(type="pil"),  # Image input
    outputs=gr.Textbox(),  # Text output (Fake or Genuine)
    live=True,  # Set to True for live feedback
    allow_flagging="never"  # Disable flagging feature completely
)

# Launch the interface and get the URL (with share=True)
interface_url = interface.launch(share=True, inline=False)

# Extract the Gradio public URL (you can use it to embed in an HTML page)
gradio_url = interface_url[0]  # URL is the first element in the tuple

# JavaScript to remove the flag button from Gradio's iframe after it loads
project_name = "Counterfiet Detection"

html_code = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Gradio Interface</title>
    <script>
        window.onload = function() {{

            var iframe = document.querySelector("iframe");
            if (iframe) {{
                var iframeDoc = iframe.contentDocument || iframe.contentWindow.document;
                setTimeout(function() {{


                    var flagButton = iframeDoc.querySelector(".gradio-flag");
                    if (flagButton) {{
                        flagButton.style.display = "none";  // Hide the flag button
                    }}
                }}, 1000);  // Wait for 1 second before hiding the flag button
            }}
        }};
    </script>
</head>
<body>

</body>
</html>
"""

# Display the HTML content
display(HTML(html_code)) # Use the imported display function

/usr/local/lib/python3.10/dist-packages/gradio/interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://617f2caf439266cb7f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
